In [1]:
import nltk
import os
from nltk import word_tokenize,sent_tokenize,pos_tag
from nltk.tag import StanfordPOSTagger
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
java_path = "C:/Program Files/Java/jdk1.8.0_144/bin/java.exe"
os.environ['JAVAHOME'] = java_path
model='stanford-postagger-2018-02-27/models/english-left3words-distsim.tagger'
jar='stanford-postagger-2018-02-27/stanford-postagger-3.9.1.jar'
st=StanfordPOSTagger(model,jar,encoding='utf-8')
lemmatizer=WordNetLemmatizer()

c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\nltk\tag\stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [2]:
senti_words = []
f=open("D:/Lex/SentiWordNet_3.0.0_20130122.txt","r",encoding="utf-8") 
all_lines=f.readlines()
for line in all_lines:
    
    w=line.split('\t')[4]
    #print(w)
    i=w.index('#')
    #print(i)
    senti_words.append(w[:i])


In [2]:
senti_words = []
f=open("improvedsamplelexicon.txt","r",encoding="utf-8") 
all_lines=f.readlines()
for line in all_lines:
    
    w=line.split('\t')[1]
    senti_words.append(w)

In [3]:
senti_words.index('nightmare')

4800

In [4]:
neg= {"aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never","no" "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite","unfulfilled","undue","anti"}

In [5]:
boost={"absolutely":0.5, "amazingly":0.125, "awfully":0.25, "completely":0.25, "considerably":0.125,
 "decidedly":0.25 , "deeply":0.0 , "enormously":0.25 ,
 "entirely":0.5 , "especially":0.0   , "exceptionally":0.0   , "extremely":0.625   ,
 "fabulously":0.25   , "flipping":0.0   , "flippin":0.0   ,
 "fricking":0.0   , "frickin":0.0   , "frigging":0.0   , "friggin":0.0   , "fully":0.375   , "fucking":0.125   ,
 "greatly":0.125   , "hella":0.0   , "highly":0.5   , "hugely":0.25   , "incredibly":0.25   ,
 "intensely":0.0   , "majorly":0.625   , "more":0.0  , "most":0.0   , "particularly":0.125   ,
 "purely":0.0   , "quite":0.125   , "really":0.375  , "remarkably":0.125   ,
 "so":0.0   , "substantially":0.125   ,
 "thoroughly":0.625   , "totally":0.5   , "tremendously":-0.25   ,
 "uber":0.0   , "unbelievably":0.25   , "unusually":0.0   , "utterly":0.5   ,
 "very":0.25   ,"almost":0.0   , "barely":-0.375   ,"badly":-0.75 ,"hardly":-0.25, "kind of":0.0   , "kinda":0.0 ,
 "less":-0.5   , "little":-0.375   , "marginally":-0.125   , "occasionally":0.0   , "partly":0.0   ,
 "scarcely":-0.25   , "slightly":-0.25   , "somewhat":-0.125, "shoddy":-0.625, "poorly":-0.75}

In [6]:
def convert_tag(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [7]:
def find_sentiment(word,pos):
    found=0
    line_nos=[index for index, value in enumerate(senti_words) if value == word]
    lines=[all_lines[lno] for lno in line_nos]
    if len(lines)==0:
        return None
    for l in lines:
        if l[0]==pos:
            pos_score,neg_score=l.split('\t')[2],l.split('\t')[3]
            found=1
            break
        else:
            continue
    if found==1:
        return(float(pos_score)-float(neg_score))
    else:
        pos_score,neg_score=lines[0].split('\t')[2],lines[0].split('\t')[3]
        return(float(pos_score)-float(neg_score))

In [8]:
def get_sentiment(word,pos):
    second_sent=find_sentiment(word,pos)
    #print(word,second_sent)
            
    if second_sent is None:
        try:
            lemma=lemmatizer.lemmatize(word,pos)
            synsets = wn.synsets(lemma)
            synset=synsets[0]
            s=synset.name()
            i=s.index('.')
            second_sent=find_sentiment(s[:i],pos)
        except:
            second_sent=0.0
        #print(word,synset.name(),second_sent)
    return second_sent

In [9]:
nouns=open('sent_nouns.txt','r',encoding='utf-8')
nouns=nouns.read()
nouns=nouns.split()

In [10]:
punctuations = '''!@'#-+='''
def clean_tweet(t):
    for c in punctuations:
             t= t.replace(c,"")
    return t

In [59]:
tweet1='''The marginalization of the Tax Cheats which was expected post #GST will now happen in right earnest as #Ewaybill rolls out. Real benefits to most organized sector companies will be seen in FY2019'''
tweet2='''From Failed #Demonetisation to Shoddy Implementation of #GST to poorly performing #Economy to unfulfilled Promise of reduction in fuel prices to not keeping the promises of providing 2 crores jobs, BJP has wrecked a nation & has broken people’s faith '''
tweet3='''There is nothing wrong with #GST implementation but the fact is we are all losers. Injected to us by @INCIndia  Even God cannot change'''
tweet4='''So, Indian businesses are mad against GST cause now they have to pay tax and earlier they weren't?'''
tweet5='''#GST in india is total failure,it like nightmare for tax payers Indians..in my view it could be rolled out after #2019LSPolls after corrections ~ Dr Subramanian @Swamy39 Speaking at 14th Annual 🇮🇳India Business Conference on ‘Indian Growth @Columbia_Biz '''
tweet6='''Now, GST leaves homestays baffled . '''

In [60]:
A=word_tokenize(clean_tweet(tweet6.lower()))
bigrams=nltk.ngrams(A,2)

In [61]:
i=0
not_found=set()
score=0.0
for b in bigrams:
    i+=1
    b_tag=st.tag(b)
    print(b_tag)
    second_gram=b_tag[1]
    first_gram=b_tag[0]
    if second_gram[0] in nouns or second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VBG') or second_gram[1].startswith('VBN') or second_gram[1].startswith('VBD'):
    #if second_gram[0] in nouns or second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VB'):
        pos2=convert_tag(second_gram[1])
        pos1=convert_tag(first_gram[1])
        
        if first_gram[0] in boost:
                score+=boost[first_gram[0]]+get_sentiment(second_gram[0],pos2)
                print('Boost:',b_tag,score)
            
   
        if first_gram[0] in neg:
                sec_score=get_sentiment(second_gram[0],pos2)
                print(second_gram[0],sec_score)
                if sec_score ==0.0 or sec_score==None:
                    score+=(get_sentiment(first_gram[0],pos1)+sec_score)
                else:
                    score+=-(sec_score)
                print('Neg:',b_tag,score)
            
        elif first_gram[0] not in neg and first_gram[0] not in boost:
            try:
                #first_score=get_sentiment(first_gram[0],pos1)
                score+=get_sentiment(second_gram[0],pos2)
                print('Normal:',b_tag,score)
            except:
                print("Not Found:",second_gram[0])
                not_found.add(second_gram[0])
    
print(score)

           

[('now', 'RB'), (',', ',')]
[(',', ','), ('gst', 'IN')]
[('gst', 'NN'), ('leaves', 'VBZ')]
[('leaves', 'VBZ'), ('homestays', 'NNS')]
[('homestays', 'NNS'), ('baffled', 'VBD')]
Normal: [('homestays', 'NNS'), ('baffled', 'VBD')] -0.25
[('baffled', 'VBN'), ('.', '.')]
-0.25


In [101]:
clean_tweet('''Can't @narendramodi govt spare #Khadi,clean fabric & the symbol of India' freedom struggle out of #GST? ''')

'Cant narendramodi govt spare Khadi,clean fabric & the symbol of India freedom struggle out of GST? '

In [11]:
def check(bigrams):
    score=0.0
    for b in bigrams:
        b_tag=st.tag(b)
        #b_tag=pos_tag(b)
        print(b_tag)
        second_gram=b_tag[1]
        first_gram=b_tag[0]
        if second_gram[0] in nouns or second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VBG') or second_gram[1].startswith('VBN') or second_gram[1].startswith('VBD'):
            pos2=convert_tag(second_gram[1])
            pos1=convert_tag(first_gram[1])
        
            if first_gram[0] in boost:
                    score+=boost[first_gram[0]]+get_sentiment(second_gram[0],pos2)
                    #print('Boost:',b_tag,score)
            
   
            if first_gram[0] in neg:
                sec_score=get_sentiment(second_gram[0],pos2)
                print(second_gram[0],sec_score)
                if sec_score ==0.0 or sec_score==None:
                    try:
                        x=get_sentiment(first_gram[0],pos1)
                        score+=(x+sec_score)
                    except:
                        print(x)
                else:
                    score+=-(sec_score)
                print('Neg:',b_tag,score)
            
            elif first_gram[0] not in neg and first_gram[0] not in boost:
                try:
                    score+=get_sentiment(second_gram[0],pos2)
                    #print('Normal:',b_tag,score)
                except:
                    print("Not Found:",second_gram[0])
                    #not_found.add(second_gram[0])
    return score
    
    

In [12]:
fw=open("176tweets.txt","r",encoding="utf-8")
rw=open("stan_tweet_result.txt","a",encoding='utf-8')
tw=fw.read().split('\n')
i=0
for t in tw:
    i+=1
    score=0.0
    tok_tweet=word_tokenize(clean_tweet(t.lower()))
    bigrams=nltk.ngrams(tok_tweet,2)
    score=check(bigrams)
    rw.write(str(t)+"  :"+str(score)+"\n")
    print(i)
fw.close()
rw.close()

[('now', 'RB'), (',', ',')]
[(',', ','), ('gst', 'IN')]
[('gst', 'NN'), ('leaves', 'VBZ')]
[('leaves', 'VBZ'), ('homestays', 'NNS')]
[('homestays', 'NNS'), ('baffled', 'VBD')]
[('baffled', 'VBN'), ('.', '.')]
1


In [442]:
word='loot'
lemma = lemmatizer.lemmatize(word)            
synsets = wn.synsets(lemma)
synset = synsets[0]
swn_synset = swn.senti_synset(synset.name())
print(swn_synset.pos_score()) 
print(swn_synset.neg_score())

0.0
0.0


In [443]:
lemma

'loot'

In [444]:
s=synset.name()
i=s.index('.')
s[:i]

'loot'

In [445]:
if word in senti_words:
    line_nos=[index+1 for index, value in enumerate(senti_words) if value == word]
    line_no=all_lines[senti_words.index(word)]
    pos_score,neg_score=line_no.split('\t')[2],line_no.split('\t')[3]
            

elif lemma in senti_words: 
    line_no=lines[senti_words.index(lemma)]
    pos_score,neg_score=line_no.split('\t')[2],line_no.split('\t')[3]
    

In [250]:
[index+1 for index, value in enumerate(senti_words) if value == 'mad']

[14272]

In [252]:
line_no

'a\t02572038\t0.375\t0.25\tmad#4 insane#2 harebrained#1\tvery foolish; "harebrained ideas"; "took insane risks behind the wheel"; "a completely mad scheme to build a bridge between two mountains"\n'

In [523]:
a='''Demonetization + GST is the biggest reset Indian Economy has witnessed since 1991. Yes...disruptive, chaotic but was necessary '''
a=word_tokenize(a.lower())

pos_tag(a)

[('demonetization', 'NN'),
 ('+', 'NNP'),
 ('gst', 'NN'),
 ('is', 'VBZ'),
 ('the', 'DT'),
 ('biggest', 'JJS'),
 ('reset', 'NN'),
 ('indian', 'JJ'),
 ('economy', 'NN'),
 ('has', 'VBZ'),
 ('witnessed', 'VBN'),
 ('since', 'IN'),
 ('1991.', 'CD'),
 ('yes', 'NNS'),
 ('...', ':'),
 ('disruptive', 'NN'),
 (',', ','),
 ('chaotic', 'JJ'),
 ('but', 'CC'),
 ('was', 'VBD'),
 ('necessary', 'JJ')]